In [10]:
from crp.attribution import CondAttribution
from crp.concepts import ChannelConcept
from crp.helper import get_layer_names
from crp.visualization import FeatureVisualization
from crp.image import plot_grid

from zennit.composites import EpsilonPlusFlat
from zennit.canonizers import SequentialMergeBatchNorm
import torch.nn as nn
import torch

from model import get_models_and_path
from data_loader import TileLoader
import os
import pandas as pd

In [16]:
models = get_models_and_path()
loader = TileLoader()

<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>


In [19]:
model = models[0][0]

# we need to set the flag so the grad calculations later work properly
for param in model.parameters():
    param.requires_grad = True
    
for param in model.gene1.parameters():
    param.requires_grad = True
    
for param in model.pretrained.parameters():
    param.requires_grad = True

In [12]:
data_dir = "../results/"
patients = [os.path.basename(f) for f in os.scandir(data_dir) if f.is_dir()]
patients.remove('p021')
filenames = []
for patient in patients:
    filenames.append(data_dir + patient + "/RUBCNL_results.csv")

df = pd.read_csv(filenames[0])
def cut_path(x):
    return x[3:]
new_col = df.path
new_col = new_col.apply(cut_path)
df.path = new_col

loader = TileLoader()

In [13]:
def get_sorted_values_by_col(df, min, max, colname='../models/res18/RUBCNL_Res18/Res18_1000_ep_29.pt', gene="RUBCNL"):
    idx = df[gene].sort_values()[min:max].index.values

    vals_orig_by_range = df.iloc[idx]
    diff = vals_orig_by_range[gene] - vals_orig_by_range[colname]

    diff = diff.abs().sort_values()
    return df.iloc[diff.index]
small_vals  = get_sorted_values_by_col(df, 0, 100)
middle_vals = get_sorted_values_by_col(df, int(len(df)/2-50), int(len(df)/2+50))
big_vals    = get_sorted_values_by_col(df, -100, len(df))

In [14]:
best = [small_vals[0:5], middle_vals[0:5], big_vals[0:5]]
worst = [small_vals[-5:], middle_vals[-5:], big_vals[-5:]]
# interesting:
interesting = []
interesting.append(best[0].index.values[1])
#set = worst[2]
#id = set.index.values[1]
tile_set = best[2]
tile_id = tile_set.index.values[1]
data = loader.open(df.path[tile_id]).unsqueeze(0)

In [20]:
# define LRP rules and canonizers in zennit
composite = EpsilonPlusFlat([SequentialMergeBatchNorm()])

# load CRP toolbox
attribution = CondAttribution(model)

# here, each channel is defined as a concept
# or define your own notion!
cc = ChannelConcept()

# get layer names of Conv2D and MLP layers
layer_names = get_layer_names(model, [nn.Conv2d, nn.Linear])

# get a conditional attribution for channel 50 in layer features.27 wrt. output 1
conditions = [{'features.27': [50], 'y': [1]}]

attr = attribution(data, conditions, composite, record_layer=layer_names)

# heatmap and prediction
attr.heatmap, attr.prediction
# activations and relevances for each layer name
attr.activations, attr.relevances

# relative importance of each concept for final prediction
rel_c = cc.attribute(attr.relevances['features.40'])
# most relevant channels in features.40
concept_ids = torch.argsort(rel_c, descending=True)

RuntimeError: can't retain_grad on Tensor that has requires_grad=False

In [ ]:


# define which concept is used in each layer
layer_map = {name: cc for name in layer_names}

# compute visualization (it takes for VGG16 and ImageNet testset on Titan RTX 30 min)
fv = FeatureVisualization(attribution, dataset, layer_map)
fv.run(composite, 0, len(dataset))

# visualize MaxRelevance reference images for top-5 concepts
ref_c = fv.get_max_reference(concept_ids[:5], 'features.40', 'relevance', composite=composite)

plot_grid(ref_c)

In [ ]:
model = ""
loader = ""
for tile in loader:
    generate_heatmap_to_file(tile)


In [ ]:
canvas = "blank"
for tile in folder:
    canvas[a,b] = tile

In [ ]:
# plot canvas

In [ ]:
# plot canvas after normalize